# Watershedding on a TYX shape image


Import packages to open and view tiffile, other python packages to deal with loading images from a certain directory. For writting results we need to create another directory, this is done by pathlib. Local function definations are included in the utils.py file which is imported along with other python packages

In [1]:
from tifffile import imread
import matplotlib.pyplot as plt
import os
import sys
from utils import NormalizeFloat, save_tiff_imagej_compatible,WatershedImage
from utils import MedianFilter, LocalThreshold3D, LocalThreshold2D,OtsuThreshold2D, Canny3D, Canny,BackGroundCorrection3D,BackGroundCorrection2D
from utils import MaxProjection, VarianceFilterTime, MidSlices, normalizeZeroOne, showImageNapari,Embryo_plot, SelectSlice, BinaryDilation
import glob
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError, AttributeError):
        from pathlib2 import Path

try:
        import tempfile
        tempfile.TemporaryDirectory
except (ImportError, AttributeError):
       from backports import tempfile
from scipy import ndimage as ndi
from skimage.filters import roberts, sobel, scharr, prewitt, gaussian
from scipy.ndimage import gaussian_filter
from skimage.morphology import watershed
from skimage.feature import peak_local_max, canny

from skimage.segmentation import find_boundaries,find_boundaries, relabel_sequential
from scipy.ndimage.morphology import distance_transform_edt, binary_fill_holes, binary_dilation
from skimage.exposure import rescale_intensity
from skimage import data, io


from skimage.morphology import remove_small_objects, binary_erosion

#!pip install napari
import napari


Now specify the source directory path, results directory path. Even if the results directory does not exist we can create one using Pathlib functions. The os function joins all the tif files found in the source directory. Axes keyword is used to specify the axis of the image we are interested to write in as a tiff file

In [2]:
sourcedir = '/Users/aimachine/Documents/OzEmbryo/'
resultsdir = '/Users/aimachine/Documents/OzEmbryo/Results/'
Path(resultsdir).mkdir(exist_ok = True)
Raw_path = os.path.join(sourcedir, '*tif')
X = glob.glob(Raw_path)
axes = 'TYX'

Write the names of the tif files found in the directory and the shape of images. Shape is a keyword in Python used to indicate the dimensions along each axis, For example it could be 100 time points, 3 channels, and 500 by 400 XY dimensions. In such case the shape would be (100,3,500,400) (Be careful about XY axes, sometimes they may appear be YX instead of XY, so a rotated image of the original image)

In [3]:
All4Dimages = []
All5Dimages = []
All3Dimages = []
for fname in X:

    image = imread(fname)
    if len(image.shape) == 3:
        All3Dimages.append(image)
    if len(image.shape) == 4:
        All4Dimages.append(image)
    if len(image.shape) > 4:
        All5Dimages.append(image)
    
    print(fname, image.shape, len(image.shape), len(All3Dimages), len(All4Dimages), len(All5Dimages))

/Users/aimachine/Documents/OzEmbryo/C2-ReCompaction_e2_2.tif (121, 792, 901) 3 1 0 0
/Users/aimachine/Documents/OzEmbryo/C2-CaFree_z1.tif (121, 941, 941) 3 2 0 0


# Always a good idea to Normalize images before doing morphological operations that depend on the image intensity

1) The percentile based normalization code is taken from csbdeep repo and added as a function in the utils directory which we imported in the first cell


Choose the image with TZCYX shape and choose a channel, do max projection along Z (axis = 1), making a time lapse image.

In [4]:
idx = 0

channel = 1

if len(All5Dimages) > 0:
 FiveDimage = All5Dimages[idx]
 print(FiveDimage.shape)

 #Select Central Z region to do the max projection

 FourDimage = SelectSlice(FiveDimage, axis = 1, slicenumber = 23)

 print(FourDimage.shape)
 ThreeDimage = FourDimage[:,0,channel,:]
 print(ThreeDimage.shape)
 TotalTimePoints = ThreeDimage.shape[0]  
 #Image shape now is TYX
 ThreeDimage = NormalizeFloat(ThreeDimage, 1, 99.8)
 for i in range(0,(ThreeDimage.shape[0])):
        ThreeDimage[i,:,:] =  normalizeZeroOne(ThreeDimage[i,:,:])

IndexError: list index out of range

In [5]:
#If you only have 3D image else skip this block

idx = 0
if len(All3Dimages) > 0:
 ThreeDimage = All3Dimages[idx]
 TotalTimePoints = ThreeDimage.shape[0] 
 ThreeDimage = NormalizeFloat(ThreeDimage, 1, 99.8)
 for i in range(0,(ThreeDimage.shape[0])):
        ThreeDimage[i,:,:] =  normalizeZeroOne(ThreeDimage[i,:,:])
        
 print(ThreeDimage.shape)        

(121, 792, 901)


# PreProcessing of images is always essential to getting a good segmentation.
1) Do BackGround correction, apply a 'huge' sigma to create a dark image and then subtract this image from the original image, this is implemented in a function we define in the utils directory which is imported in the first block


2) Do Median filtering on the resulting image with 'small' sigma to create a edge-preserved image

In [18]:
blur_radius = (25,25)

Size_remove = 1000
TwoDCorrectedThreeDimage = np.zeros([ThreeDimage.shape[0],ThreeDimage.shape[1], ThreeDimage.shape[2]]) 
TwoDMedianThreeDimage = np.zeros([ThreeDimage.shape[0],ThreeDimage.shape[1], ThreeDimage.shape[2]]) 
TwoDBinaryThreeDimage = np.zeros([ThreeDimage.shape[0],ThreeDimage.shape[1], ThreeDimage.shape[2]]) 
TwoDDilatedThreeDimage = np.zeros([ThreeDimage.shape[0],ThreeDimage.shape[1], ThreeDimage.shape[2]]) 
CannyThreeDimage = np.zeros([ThreeDimage.shape[0],ThreeDimage.shape[1], ThreeDimage.shape[2]]) 
median_filter_radius = (8,8)
for j in range(0, ThreeDimage.shape[0]):
    
    TwoDCorrectedThreeDimage[j,:] = BackGroundCorrection2D(ThreeDimage[j,:], blur_radius) 
    TwoDMedianThreeDimage[j,:] = MedianFilter(TwoDCorrectedThreeDimage[j,:], median_filter_radius)
    
    TwoDBinaryThreeDimage[j,:] = OtsuThreshold2D(TwoDMedianThreeDimage[j,:], size = Size_remove)
    CannyThreeDimage[j,:] = Canny(TwoDBinaryThreeDimage[j,:], sigma = 3)
    TwoDDilatedThreeDimage[j,:] = BinaryDilation(TwoDBinaryThreeDimage[j,:], iterations = 20)
   


In [19]:
#close the viewer to procees next

showImageNapari(ThreeDimage,TwoDDilatedThreeDimage)


In [20]:
#Save the Binaryimages as ImageJ compatiable tiff files
save_tiff_imagej_compatible((resultsdir + "2DBlur" + '.tif') ,TwoDDilatedThreeDimage, axes)

/Users/aimachine/BioPython/utils.py:371: UserWarning: Converting data type from 'float64' to ImageJ-compatible 'float32'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))
/Users/aimachine/BioPython/utils.py:242: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]


# Do Distance transform on 2D slice by slice blurred image

In [21]:
TotalTimePoints = ThreeDimage.shape[0] 

TwoDBlurThreeDdistance = np.zeros([TwoDDilatedThreeDimage.shape[0],TwoDDilatedThreeDimage.shape[1], TwoDDilatedThreeDimage.shape[2]]) 


for j in range(0, TotalTimePoints):
 
 TwoDBlurThreeDdistance[j,:] = ndi.distance_transform_edt(np.logical_not((CannyThreeDimage[j,:])))  


In [22]:
showImageNapari(ThreeDimage,TwoDBlurThreeDdistance)

In [23]:
#Save the slice by slice distance transformed image in results directory
save_tiff_imagej_compatible((resultsdir + "2DBlurDistTransform" + '.tif') ,TwoDBlurThreeDdistance, axes)

/Users/aimachine/BioPython/utils.py:371: UserWarning: Converting data type from 'float64' to ImageJ-compatible 'float32'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))
/Users/aimachine/BioPython/utils.py:242: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]


# 2D watershed

In [ ]:
Time = []
Embryos = []
Seed_dist = 50
labels = np.zeros([TwoDBlurThreeDdistance.shape[0],TwoDBlurThreeDdistance.shape[1], TwoDBlurThreeDdistance.shape[2]])
for j in range(0, TotalTimePoints):
 labels[j,:] = WatershedImage(TwoDBlurThreeDdistance[j,:], 5,5, min_distance = Seed_dist ) 
 Time.append(j)
 Embryos.append(np.amax(labels[j,:]) - np.amin(labels[j,:]) ) 

Embryo_plot(Time, Embryos)

In [25]:
showImageNapari(ThreeDimage,labels)

In [26]:
save_tiff_imagej_compatible((resultsdir + "TwoDimensionalWatershed" + '.tif') ,  labels, axes)


/Users/aimachine/BioPython/utils.py:371: UserWarning: Converting data type from 'float64' to ImageJ-compatible 'float32'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))
/Users/aimachine/BioPython/utils.py:242: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]
